In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [2]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [4]:
import tmdbsimple as tmdb
tmdb.API_KEY = '5e3a0224faa49e33a468c4948a547e2c'

In [5]:
from imdb import IMDb
ia = IMDb()

In [6]:
import rotten_tomatoes_client as rtc

In [ ]:
train_set['release_date']=pd.to_datetime(train_set['release_date'], format='%m/%d/%y')
test_set['release_date']=pd.to_datetime(test_set['release_date'], format='%m/%d/%y')
train_set['release_year']=train_set['release_date'].dt.year
train_set['release_month']=train_set['release_date'].dt.month
test_set['release_year']=test_set['release_date'].dt.year
test_set['release_month']=test_set['release_date'].dt.month
test_set.at[828, 'release_month'] = 5

In [ ]:
def fixyear(setname, colname='release_year'):
    for i in range(len(setname[colname])):
        try:
            s_result = ia.get_movie(setname.imdb_id[i][2:])
            the_unt = s_result
            ia.update(the_unt)
            year = the_unt['year']
            if setname.release_year[i]!=year:
                setname.at[i,'release_year']=year
        except:
            try:
                result = rtc.RottenTomatoesClient.search(term='Whiplash')

                for l in range(len(result)):
                    if eval(setname.loc[setname.title==setname.title[i]].cast.tolist()[0])[0]['name'] in result['movies'][l]['subline'].split(','):
                        ix=l
                year=result['movies'][ix]['year']
                if setname.release_year[i]!=year:
                    setname.at[i,'release_year']=year
            except Exception as e:
                print(setname.title[i])
                print(e)

In [ ]:
fixyear(train_set)
fixyear(test_set)

In [ ]:
train_set.release_year.hist()

In [ ]:
def weightedrating (setname, colname='title'):
    rating=[]
    titlelist = setname.title.tolist()
    for i in range(len(setname[colname])):
        counter = titlelist.count(setname.title[i])
        if counter==1:
            try:
                result = rtc.RottenTomatoesClient.search(term=setname[colname][i])
                rating.append(result['movies'][0]['meterScore']/10)
            except:
                try:
                    s_result = ia.get_movie(setname.imdb_id[i][2:])
                    the_unt = s_result
                    ia.update(the_unt)
                    rating.append(the_unt['rating'])
                except:
                    try:
                        movie_id = setname.loc[setname.title==setname[colname][i]]['id'].tolist()[0]
                        movie = tmdb.Movies(movie_id)
                        response = movie.info()
                        rating.append(movie.vote_average)
                    except:
                        try:
                            s_result = ia.search_movie(setname[colname][i])
                            the_unt = s_result[l]
                            ia.update(the_unt)
                            rating.append(the_unt['rating'])
                        except:
                            try:
                                search = tmdb.Search()
                                response = search.movie(query=setname[colname][i])
                                for l in range(10):
                                    try:
                                        if search.results[l]['id']==setname.at[i,'id']:
                                            rating.append(search.results[0]['vote_average'])
                                    except:
                                        break
                            except Exception as e:
                                print(titles)
                                print(e)
        else:
            try:
                s_result = ia.get_movie(setname.imdb_id[i][2:])
                the_unt = s_result
                ia.update(the_unt)
                rating.append(the_unt['rating'])
            except:
                try:
                    movie_id = setname.loc[setname.title==setname[colname][i]]['id'].tolist()[0]
                    movie = tmdb.Movies(movie_id)
                    response = movie.info()
                    rating.append(movie.vote_average)
                except:
                    try:
                        s_result = ia.search_movie(setname[colname][i])
                        the_unt = s_result[l]
                        ia.update(the_unt)
                        rating.append(the_unt['rating'])
                    except:
                        try:
                            search = tmdb.Search()
                            response = search.movie(query=setname[colname][i])
                            for l in range(10):
                                try:
                                    if search.results[l]['id']==setname.at[i,'id']:
                                        rating.append(search.results[0]['vote_average'])
                                except:
                                    continue
                            
                        except Exception as e:
                            print(titles)
                            print(e)
    return rating     

In [ ]:
train_rating = weightedrating(train_set)

In [ ]:
test_rating = weightedrating(test_set)

In [ ]:
train_set['rating'] = train_rating
test_set['rating'] = test_rating

In [7]:
test_set.at[2398,'rating'] = 5.4

In [ ]:
f = open('C:/Users/m-sim/Desktop/train_set.pkl', 'wb')
pickle.dump(train_set, f)
f.close()
f = open('C:/Users/m-sim/Desktop/test_set.pkl', 'wb')
pickle.dump(test_set, f)
f.close()

In [7]:
f = open('C:/Users/m-sim/Desktop/train_set.pkl','rb')
train_set = pickle.load(f)
f.close()
f = open('C:/Users/m-sim/Desktop/test_set.pkl','rb')
test_set = pickle.load(f)
f.close()

In [8]:
test_set.at[828, 'release_month'] = 5

In [7]:
f = open('C:/Users/m-sim/Desktop/variables/train_year_rating.pkl', 'wb')
pickle.dump([train_set.release_year.tolist(),train_set.rating.tolist()], f)
f.close()
f = open('C:/Users/m-sim/Desktop/variables/test_year_rating.pkl', 'wb')
pickle.dump([test_set.release_year.tolist(),test_set.rating.tolist()], f)
f.close()

In [ ]:
response = urlopen('https://www.rottentomatoes.com/m/whiplash_2014')

soup = bs(response, 'lxml')

aud_rating = soup.find("span", {"class": "mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience"}).get_text(strip=True)
aud_rating = int(aud_rating.split('%')[0])/10
aud_rating

In [ ]:
result = rtc.RottenTomatoesClient.search(term='rocky')
result['movies'][0]['url']

In [32]:
import string

def aud_rate(setname,colname='title'):
    aud_rating_list=[]
    for i in range(len(setname[colname])):
        try:
            title = '_'.join(str(setname[colname][i]).lower().translate(str.maketrans('', '', string.punctuation)).split(' '))
            response = urlopen('https://www.rottentomatoes.com/m/'+title)

            soup = bs(response, 'lxml')

            aud_rating = soup.find("span", {"class": "mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience"}).get_text(strip=True)
            aud_rating = int(aud_rating.split('%')[0])/10
            aud_rating_list.append(aud_rating)
        except:
            try:
                title = '_'.join(str(setname[colname][i]).lower().translate(str.maketrans('', '', string.punctuation)).split(' '))
                year = str(setname.loc[setname.title==setname[colname][i]]['release_year'].tolist()[0])
                ty = '_'.join([title,year])
                response = urlopen('https://www.rottentomatoes.com/m/'+ty)

                soup = bs(response, 'lxml')

                aud_rating = soup.find("span", {"class": "mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience"}).get_text(strip=True)
                aud_rating = int(aud_rating.split('%')[0])/10
                aud_rating_list.append(aud_rating)
            except:
                try:
                    result = rtc.RottenTomatoesClient.search(term=setname[colname][i])
                    for l in range(int(len(result)/10)):
                        if eval(setname.loc[setname.title==setname.title[i]].cast.tolist()[0])[0]['name'] in result['movies'][l]['subline'].split(','):
                            response = urlopen('https://www.rottentomatoes.com'+result['movies'][l]['url'])
                            soup = bs(response, 'lxml')

                            aud_rating = soup.find("span", {"class": "mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience"}).get_text(strip=True)
                            aud_rating = int(aud_rating.split('%')[0])/10
                            aud_rating_list.append(aud_rating)
                        elif setname.at[i,'release_year']==result['movies'][l]['year']:
                            response = urlopen('https://www.rottentomatoes.com'+result['movies'][l]['url'])
                            soup = bs(response, 'lxml')

                            aud_rating = soup.find("span", {"class": "mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience"}).get_text(strip=True)
                            aud_rating = int(aud_rating.split('%')[0])/10
                            aud_rating_list.append(aud_rating)
                        else:
                            aud_rating_list.append(setname.loc[setname.title==setname[colname][i]]['rating'].tolist()[0])
                except:
                    print(setname[colname][i])
                    print(i)
                    aud_rating_list.append(setname.loc[setname.title==setname[colname][i]]['rating'].tolist()[0])
    return aud_rating_list

In [33]:
train_aud_rate =  aud_rate(train_set)

And Here's What's Happening to Me
95
Sällskapsresan - eller finns det svenskt kaffe på grisfesten?
218
Gary Gulman: It's About Time
224
The National Shotgun
237
Would I Lie to You? 2
250
Delhi Dance
253
Voracious
254
The Inhabited Island 2: Rebellion
295
Asterix & Obelix Take on Caesar
367
Dear Guest, When Will You Leave?
419
The Book of Mormon Movie, Volume 1: The Journey
470
Nikitich and The Dragon
487
Alaipayuthey
516
El pico 2
564
Katyar Kaljat Ghusali
586
А поутру они проснулись
591
A Prince (almost) Charming
595
The Day After Tomorrow
680
Ivan Tsarevich & the Grey Wolf 2
884
In the Forests of Siberia
911
O Kadhal Kanmani
918
Pokémon 4Ever: Celebi - Voice of the Forest
943
Fuck You Goethe 2
960
The Broken Hearts Club: A Romantic Comedy
981
Zyzzyx Road
1006
Billy Gardell: Halftime
1007
Stepfather II: Make Room For Daddy
1046
FBI: Frikis buscan incordiar
1053
Crazy About Ya
1092
El robobo de la jojoya
1100
Pokémon: Arceus and the Jewel of Life
1136
Candy Razors
1186
Atlas Shrugged P

In [34]:
test_set.at[2398, 'title'] = 'GUSKÔ BUDORI NO DENKI'
test_set.at[3628, 'title'] = 'Barefoot'
test_set.at[2425, 'title'] = 'La vérité si je mens! 3'
test_set.at[33, 'title'] = 'Ayurveda: The Art of Being'

In [35]:
test_aud_rate = aud_rate(test_set)

Cycling with Molière
116
House III: The Horror Show
134
Chelovek, kotoryy znal vsyo
165
Puella Magi Madoka Magica the Movie Part III: Rebellion
217
La caliente niña Julietta
243
Roadgames
298
Pokémon Heroes: Latios and Latias
310
French Women
362
Æon Flux
366
Hooked on the Game
400
Cats & Dogs 2 : The Revenge of Kitty Galore
405
Zombie Fever
445
Iron & Blood: The Legend of Taras Bulba
492
Yu-Gi-Oh! The Movie
539
Ticket to Vegas
542
A Nightmare on Elm Street Part 2: Freddy's Revenge
588
Italiano medio
601
The Second Jungle Book: Mowgli & Baloo
610
Ballplayer
629
The Tobacconist of Vallecas
633
Garv: Pride and Honour
636
Eternal Summer
691
Pokémon the Movie: Genesect and the Legend Awakened
699
Irudhi Suttru
713
Jails, Hospitals & Hip-Hop
828
Nasty Old People
844
Lights of New York
875
For Greater Glory - The True Story of Cristiada
884
Winnebago Man
915
F.C. De Kampioenen: Kampioen zijn blijft plezant
939
Erkan & Stefan 2
973
Escape to Victory
988
Shikshanachya Aaicha Gho
1073
Willy/Mil

In [36]:
print(len(train_aud_rate))
print(len(train_set))
print(len(test_aud_rate))
print(len(test_set))

3000
3000
4398
4398


In [37]:
f = open('C:/Users/m-sim/Desktop/variables/train_aud_rate.pkl', 'wb')
pickle.dump(train_aud_rate, f)
f.close()
f = open('C:/Users/m-sim/Desktop/variables/test_aud_rate.pkl', 'wb')
pickle.dump(test_aud_rate, f)
f.close()

In [38]:
train_set['aud_rating'] = train_aud_rate
test_set['aud_rating']  = test_aud_rate

In [ ]:
def runtimefix(setname, colname='title'):
    meantime = float(format(np.mean(setname.runtime),'.3f'))
    for i in range(len(setname[colname])):
        if pd.isnull(setname.runtime[i]):
            try:
                s_result = ia.get_movie(setname.at[i,'imdb_id'][2:])
                the_unt = s_result[0]
                ia.update(the_unt)
                setname.at[i,'runtime'] = the_unt['runtime']
            except:
                setname.at[i,'runtime'] =  meantime

In [ ]:
runtimefix(train_set)
runtimefix(test_set)

In [ ]:
def budgetfix(setname, colname='imdb_id'):
    mean = np.mean(setname.budget)
    for imdbid in setname[colname]:
        if setname.loc[setname[colname]==imdbid].budget.tolist()[0]==0:
            print(imdbid)
            row_index = setname.loc[setname[colname]==imdbid].index.tolist()[0]
            try:
                response = urlopen('https://www.imdb.com/title/'+imdbid)
                soup = bs(response, 'xml')
                budget = soup.find("span", "attribute")
                budget = budget.parent.get_text(strip=True)
                budget = int(''.join([i for i in budget if i.isdigit()]))
                setname.at[row_index,'budget'] = budget
            except Exception as e:
                setname.at[row_index,'budget'] = mean

In [ ]:
import pickle
f = open('D:/Study/Kaggle/tmdb-box-office-prediction/train_set.pkl', 'wb')
pickle.dump(train_set, f)
f.close()
f = open('D:/Study/Kaggle/tmdb-box-office-prediction/test_set.pkl', 'wb')
pickle.dump(test_set, f)
f.close()

In [ ]:
# def belongs_to_col(setname,colname):
#     collectionlist = setname[colname].tolist()

#     newcollection = []
#     for collection in collectionlist:
#         if pd.isnull(collection):
#             newcollection.append('nocollection')
#         else:
#             newcollection.append(collection.split(',')[1].split(':')[1].lstrip().replace("'",''))


#     setname[colname]=newcollection

# belongs_to_col(train_set,'belongs_to_collection')
# belongs_to_col(test_set,'belongs_to_collection')

# train_set = pd.concat([train_set,pd.get_dummies(train_set['belongs_to_collection'])],axis=1)
# train_set = train_set.drop(['belongs_to_collection','nocollection'],axis=1)

# test_set = pd.concat([test_set,pd.get_dummies(test_set['belongs_to_collection'])],axis=1)
# test_set = test_set.drop(['belongs_to_collection','nocollection'],axis=1)

In [ ]:
def belongscol(setname,colname='belongs_to_collection'):
    collist=[]
    for col in setname[colname]:
        if pd.isnull(col):
            collist.append(0)
        else:
            collist.append(1)
    return collist

train_set['belongs_to_collection'] = belongscol(train_set)
test_set['belongs_to_collection'] = belongscol(test_set)

In [ ]:
genrelist = train_set['genres'].tolist()

newgenre = []
for genre in genrelist:
    if pd.isnull(genre):
        newgenre.append(['nogenre'])
    else:
        genres=[]
        for diffgenres in eval(genre):
            genres.append(diffgenres['name'])
        newgenre.append(genres)
train_set['genres'] = newgenre
genrelist = train_set['genres'].tolist()

genrename = []
for genres in genrelist:
    if genres=='nogenre':
        if genres not in genrename:
            genrename.append(genres)
    else:
        for genre in genres:
            if genre not in genrename:
                genrename.append(genre)

fullonehot=[]
for genres in genrelist:
    onehotlists=[]
    for genre in genres:
        onehot=[]
        for genre_name in genrename:
            if genre==genre_name:
                onehot.append(1)
            else:
                onehot.append(0)
        onehotlists.append(onehot)
    fullonehot.append(np.bitwise_or.reduce(onehotlists))

onehotdf = pd.DataFrame(fullonehot)
onehotdf.columns=genrename
train_set = pd.concat([train_set,onehotdf],axis=1)
train_set = train_set.drop(['genres','nogenre'],axis=1)

In [ ]:
def dictionarycol(setname,colname):
    collist = setname[colname].tolist()

    newlist = []
    for col in collist:
        if pd.isnull(col):
            newlist.append(['none'])
        else:
            names=[]
            for diffcol in eval(col):
                names.append(diffcol['name'])
            newlist.append(names)
    setname[colname] = newlist
    newlist = setname[colname].tolist()
    
    savedname = []
    for name in newlist:
        if name=='none':
            if name not in savedname:
                savedname.append(name)
        else:
            for names in name:
                if names not in savedname:
                    savedname.append(names)
    fullonehot=[]
    for name in newlist:
        onehotlists=[]
        for names in name:
            onehot=[]
            for sname in savedname:
                if names==sname:
                    onehot.append(1)
                else:
                    onehot.append(0)
            onehotlists.append(onehot)
        fullonehot.append(np.bitwise_or.reduce(onehotlists))
    onehotdf = pd.DataFrame(fullonehot)
    onehotdf.columns=savedname
    return onehotdf

In [ ]:
testonehotdf = dictionarycol(test_set,'genres')
test_set = pd.concat([test_set,onehotdf],axis=1)
test_set = test_set.drop(['genres'],axis=1)


# onehotdf = dictionarycol(train_set,'production_companies')
# testonehotdf = dictionarycol(test_set,'production_companies')

# train_set = pd.concat([train_set,onehotdf],axis=1)
train_set = train_set.drop(['production_companies'],axis=1)
# test_set = pd.concat([test_set,onehotdf],axis=1)
test_set = test_set.drop(['production_companies'],axis=1)

In [ ]:
def production_country(setname,colname):
    collist = setname[colname].tolist()

    newlist = []
    for col in collist:
        if pd.isnull(col):
            newlist.append('none')
        else:
            names=[]
            for diffcol in eval(col):
                names.append(diffcol['name'])
            newlist.append(names[0])
    setname[colname] = newlist
    newlist = setname[colname].tolist()

production_country(train_set,'production_countries')
production_country(test_set,'production_countries')

In [ ]:
def countrytonum (setname,colname='production_countries'):
    vallist=[]
    for val in setname[colname]:
        if val!='United States of America':
            vallist.append(1)
        else:
            vallist.append(2)
    return vallist

In [ ]:
train_set['production_countries'] = countrytonum(train_set)
test_set['production_countries'] = countrytonum(test_set)

In [ ]:
def director(setname,colname):

    collist = setname[colname].tolist()

    newlist = []
    
    for col in collist:
        if pd.isnull(col):
            newlist.append('none')
        else:
            names=[]
            for diffcol in eval(col):
                if diffcol['job']=='Director':
                    names.append(diffcol['name'])
            if names==[]:
                newlist.append('none')
            else:
                newlist.append(names[0])
    setname['director'] = newlist

In [ ]:
director(train_set, 'crew')
train_set = train_set.drop(['crew'], axis=1)

In [ ]:
director(test_set,'crew')
test_set = test_set.drop(['crew'], axis=1)

In [ ]:
# train_set = pd.concat([train_set,pd.get_dummies(train_set['director'])],axis=1)
# test_set = pd.concat([test_set,pd.get_dummies(test_set['director'])],axis=1)

In [ ]:
def cast(setname, colname):
    collist = setname[colname].tolist()

    newlist = []
    for col in collist:
        if col=='[]' or pd.isnull(col):
            newlist.append('none')
        else:
            names=[]
            for diffcol in eval(col):
                names.append(diffcol['name'])
            newlist.append(names[0])
    setname[colname]=newlist

cast(train_set,'cast')
cast(test_set,'cast')

In [ ]:
# train_set = pd.concat([train_set,pd.get_dummies(train_set['cast'])],axis=1)
# test_set = pd.concat([test_set,pd.get_dummies(test_set['cast'])],axis=1)

In [ ]:
#train_set = pd.concat([train_set,pd.get_dummies(train_set['original_language'])],axis=1)
#test_set = pd.concat([test_set,pd.get_dummies(test_set['original_language'])],axis=1)
#train_set = pd.concat([train_set,pd.get_dummies(train_set['production_countries'])],axis=1)
#test_set = pd.concat([test_set,pd.get_dummies(test_set['production_countries'])],axis=1)

In [ ]:
def lang(setname,colname):
    langlist=[]
    for lang in setname[colname]:
        if lang!='en':
            langlist.append(0)
        else:
            langlist.append(1)
    setname[colname]=langlist

lang(train_set,'original_language')
lang(test_set,'original_language')

In [ ]:
#def country(setname,colname):
#    countrylist=[]
#    for count in train_set['production_countries']:
#        if count!=['United States of America']:
#            countrylist.append(0)
#        else:
#            countrylist.append(1)
#    train_set['production_countries']=countrylist
        
#country(train_set,'production_countries')
#country(test_set,'production_countries')

In [ ]:
train_set = train_set.drop(['homepage','overview','spoken_languages','tagline','poster_path','cast','director','original_title','Keywords','status'], axis=1)
test_set = test_set.drop(['nogenre','homepage','overview','spoken_languages','tagline','poster_path','cast','director','original_title','Keywords','status'], axis=1)

In [ ]:
# train_set_col = train_set.columns.tolist()
# test_set_col = test_set.columns.tolist()

# for i in train_set_col:
#     for l in test_set_col:
#         if i==l:
#             test_set_col.remove(l)

# len(pd.concat([train_set,test_set[test_set_col]],axis=1).columns)

# train_set_col = train_set.columns.tolist()
# test_set_col = test_set.columns.tolist()

# for i in test_set_col:
#     for l in train_set_col:
#         if i==l:
#             train_set_col.remove(l)

In [ ]:
X = train_set.drop(['id','revenue','imdb_id','release_date','title'], axis=1).copy()
y = train_set['revenue'].values

In [ ]:
#X = X.apply(pd.to_numeric, errors='coerce')
X.fillna(0, inplace=True)
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=500, min_samples_leaf=5, random_state=42)
forest_reg.fit(x_train, y_train)

In [ ]:
y_pred = forest_reg.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
msle_scorer = make_scorer(mean_squared_log_error)

scores = cross_val_score(forest_reg, X, y,
                         scoring=msle_scorer, cv=10)
forrest_rmsle = np.sqrt(scores)

In [ ]:
np.mean(forrest_rmsle)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [800, 850], 'max_features': [1], 'min_samples_leaf':[1,5], 'min_samples_split':[2,5]}
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring=msle_scorer, return_train_score=True)
grid_search.fit(X, y)

In [ ]:
grid_search.best_params_

In [ ]:
np.sqrt(grid_search.cv_results_['mean_test_score'])

In [ ]:
X = test_set.drop(['id','imdb_id','release_date','title'], axis=1).copy()
X.fillna(0, inplace=True)

test_y_pred = forest_reg.predict(X)

submission_df = test_set['id'].copy()
submission_df = pd.DataFrame(submission_df)
submit_df = submission_df[:4398]
submit_df['revenue'] = test_y_pred
submit_df.to_csv('submission2.csv', index=False)